# 생산적 신경망

> 콘텐츠 및 특정 작가 스타일의 이미지를 만드는 네트워크, 이 기법을 스타일 트랜스퍼라고 한다.

> 특정 유형의 생성적 적대적 네트워크(GAN)을 이요해 새로운 사람 얼굴을 만드는 네트워크

> 언어 모델을 사용해 새롱누 문장 생성하는 네트워크

## 신경망 스타일 트랜스퍼 

스타일 트랜스퍼 알고리즘에 콘텐츠 이미지(C)와 스타일 이미지(S)를 제공해야한다. 알고리즘은 콘텐츠 이미지로부터 내용을 가져오고, 스타일 이미지로부터 스타일을 가져와 새로운 이미지 (O)를 만든다.

**CNN작동방식**

CNN이 객체 인식을 학습할 때 앞부분의 초기 레이어는 선, 곡선 및 모양과 같은 일반적인 정보를 학습한다. CNN의 뒷부분 레이어에서는 눈, 건물 및 나무와 같은 상위 개념의 이미지를 인식한다. 따라서 비슷한 이미지의 마지막 레이어의 값은 상당히 비슷한 경향을 보인다. 스타일 트랜스퍼에서는 이 개념을 콘텐츠 오차에 적용한다. 마지막 레이어에서 콘텐츠 이미지와 생성된 이미지는 유사해야한다. 그리고 MSE를 사용해 두 이미지의 유사도를 계산한다. 두 이미지 사이의 오차값을 낮추기 위해 최적화 알고리즘을 사용해야한다.

이미지 스타일은 일반적으로 그램 매트릭스라는 기술로 CNN의 여러 레이어에서 캡처된다. 그램 매트릭스는 여러 레이어에서 캡처된 피처 맵 간의 상관관계를 계산한다. 그램 매트릭스는 스타일을 계산하는 방법을 제공한다. 유사한 스타일이 적용된 이미지에 대해 그램 매트릭스에 대해 비슷한 값을 가진다. 또한 스타일 오차는 스타일 이미지와 생성된 이미지의 그램 매트릭스 사이의 MSE를 사용해 계산된다. 

다른 딥러닝 모델과 거의 비슷하지마 차이점은 오차 계산이 분류 또는 회귀 모델보다 더 복잡하다는 점이다. 

### 데이터 로딩

미리 훈련된 VGG 모델을 사용하기 때문에 사전 훈련된 VGG 모델을 훈련할 때 사용한 값으로 입력될 이미지 데이터를 정규화 해야한다.

In [ ]:
# 이미지 크기 고정
# GPU를 사용하지 않을 경우 크기를 더 줄인다.

import torch
from torchvision import transforms
from torch.autograd import Variable
from PIL import Image

imsize = 512
is_cuda = torch.cuda.is_available()

# VGG모델을 이용해 학습할 수 있도록 이미지 변환

prep = transforms.Compose([
                           transforms.Resize(imsize),
                           transforms.ToTensor(),
                           # BGR로 변환
                           transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]),
                           transforms.Normalize(
                               # 이미지넷 평균을 빼기
                               mean = [0.40760392, 0.45795686, 0.48501961],
                               std=[1,1,1]),
                           transforms.Lambda(lambda x: x.mul_(255)),
])

# 생성된 이미지를 시각화 할 수 있는 형식으로 다시 변환

postpa = transforms.Compose([
                             transforms.Lambda(lambda x: x.mul_(1./255)),
                             transforms.Normalize(
                                 # 이미지넷 평균 더하기
                                 mean = [-0.40760382, -0.45795686, -0.48501961],
                                 std = [1,1,1]),
                             # RGB로 전환
                             transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]),
])

postpb = transforms.Compose([transforms.ToPILImage()])

# 이미지 데이터가 허용 범위를 벗어나지 않도록 조정
# 결과 범위를 [0,1]로 한정
def postp(tensor):
    t = postpa(tensor)
    t[t>1] = 1
    t[t<0] = 0
    img = postpb(t)
    return img

# 데이터 로딩을 쉽게하는 유틸리티 함수
def image_loader(image_name):
    image = Image.open(image_name)
    image = Variable(prep(image))
    # 네트워크의 입력 차원을 맞추기 위한 배치 차원 조정
    image = image.unsqueeze(0)
    return image

**prep 함수**는 필요한 모든 전처리를 수행하며 VGG모델이 학습한 데이터의 값으로 입력 데이터를 정규화한다. 모델의 출력을 원래 값으로 복원하기 위해서 반대방향으로 정규화돼야한다.**postpa 함수**가 이 역할을 담당한다. 생성된 모델은 값의 허용 범위를 벗어날 수 있기 때문에 **postp 함수**로 이를 변환한다. 마지막 **image_loader** 함수는 이미지를 로드하고 전처리를 수행한 후 데이터를 Variable로 변환한다. 다음 함수는 스타일과 콘텐츠 이미지를 로드한다. 

    style_img = image_loader("Images/vangogh_starry_night.jpg")
    content_img = image_loader("Images/Tuebingen_Neckarfront.jpg")
    opt_img = Variable(content_img.data.clone(), requires_grad=True)

입력이미지가 콘텐츠 이미지와 스타일 이미지에 더 가까워지도록 조정하기 위해 옵티마이저를 사용한다. 이런 이유로 opt_img를 생성할 때 requires_grad = True로 설정한다.


In [ ]:
#구글 코랩 파일 업로드 방법
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

for fn in uploaded.keys(): # 업로드된 파일 정보 출력
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


In [ ]:
style_img = image_loader("style_img.jpeg")
content_img = image_loader("content_img.jpg")
opt_img = Variable(content_img.data.clone(), requires_grad=True)

### VGG 모델 생성
torchvisions.models에서 제공하는 사전 학습된 모델을 사용한다. 
이모델은 피처를 추출하는 용도로만 사용한다. 

VGG 모든 컨볼루션 블록은 Feature 모듈에 정의되고 전연결이나 선형레이어는 Classifier 모듈에 정의된다. VGG 모델의 가중치나 파라미터는 학습시키지 않을것이다. 따라서 모든 학습 파라미터는 고정된다.

In [ ]:
from torchvision.models import vgg19

vgg = vgg19(pretrained=True).features
# VGG 컨볼루션 블록만 가져온다.

# 모델이 학습되지 않도록 레이어 고정
for param in vgg.parameters():
    param.requires_grad = False

### 콘텐츠 오차

콘텐츠 오차는 네트워크에 2개의 임지를 입력하고 특정 레이어에서 두 이미지의 출력에 대한 MSE를 계산한다.

    target_layer = dummy_fn(content_img)
    noise_layer = dummy_fn(noise_img)
    criterion = nn.MSELoss()
    content_loss = criterion(target_layer, noise_layer)

### 스타일 오차

스타일 오차는 여러 레이어에 걸쳐 계산된다. 스타일 오차는 각 피처 맵으로 생성한 그램 매트릭스의 MSE이다. 그램 매트릭스는 피처의 상관관계 값이다. 

In [ ]:
# 매치크기, 채널, 값으로 구성된 피처 맵을 그램 행렬 계산을 하기 위해서는 
# 각 채널의 값을 1차원 배열로 펼쳐야 한다. 이렇게 만들어진 행렬을 자신의 전치 행렬과 곱해 상관관계를 구한다.
from torch import nn

class GramMatrix(nn.Module):
    def forward(self, input):
        b, c, h, w = input.size() 
        # b=배치크기, c=필터 또는 채널, h,w = 높이, 폭
        features = input.view(b, c, h*w) #배치와 채널을 그대로 유지하고 높이와 폭 차원을 1차원으로 평평하게 만든다.
        gram_matrix = torch.bmm(features, features.transpose(1,2)) #평평하게 만든 벡터와, 그 백터의 전치를 곱해 계산한다. 
        gram_matrix.div_(h*w)
        return gram_matrix

그램 매트릭스의 값을 요소 수로 나눠 정규화한다. 특정 피처맵이 스타일 오차 점수에 주도적으로 영향을 미치는 상황을 막을 수 있다. 

In [ ]:
class StyleLoss(nn.Module):
    def forward(self, inputs, targets):
        out = nn.MSELoss()(GramMatrix()(inputs), targets)
        return out

### VGG 모델 레이어의 오차 추출

스타일 오차와 콘텐츠 오차를 계산하는데 필요한 여러 컨볼루션 레이어의 오차를 추출할 수 있다.

In [ ]:
class LayerActivations():
    features = []

    def __init__(self, model, layer_nums): # register_forward_hook 메서드를 호출하는 모델과 출력 추출 대상이 되는 레이어 번호를 입력받는다.
        self.hooks = []
        for layer_num in layer_nums: # 입력받은 레이어 번호로 for문을 수행해 출력을 가져오는데 필요한 forward hook을 등록한다.
            self.hooks.append(model[layer_num].register_forward_hook(self.hook_fn))
    
    def hook_fn(self, module, input, output): #hook_fn 메서드는 출력을 캡처해 feature 배열에 저장한다.
        self.features.append(output)
    
    def remove(self): # 지정한 레이어에서 더이상 출력 추출을 원하지 않는다면, remove 메서드를 호출한다.
        for hook in self.hooks:
            hook.remove()


In [ ]:
# 스타일과 콘텐츠 이미지의 필요한 레이어의 출력을 추출하는 유틸리티 함수

def extract_layers(layers, img, model=None):
    la = LayerActivations(model, layers) # 위에서 만든 LayerActivations 객체를 만든다.
    # features 배열에 이전 실행 결과가 포함될 수 있으므로 새로운 배열ㅇ르 설정하고 모델에 이미지를 전달한다.
    la.features = []
    out = model(img)
    la.remove() # remove 메서드를 호출해 등록된 모든 후크를 모델에서 제거하고 features 배열을 반환한다.
    return la.features

In [ ]:
style_layers = [1,6,11,20,25]
content_layers = [21]
loss_layers = style_layers + content_layers

content_targets = extract_layers(content_layers, content_img, model = vgg)
style_targets = extract_layers(style_layers, style_img, model=vgg)

content_targets = [t.detach() for t in content_targets]
style_targets = [GramMatrix()(t).detach() for t in style_targets]

targets = style_targets + content_targets

대상 레이어의 출력을 추출하면, 추출된 출력은 그래프와 분리해야한다. 앞에서 추출한 모든 출력은  Variable 객체이다. 따라서 모든 출력은 자기를 만드는 모든 방법에 대한 정보를 유지하고 있다. 그러나 스타일 트랜스퍼에서는 스타일 이미지나 콘텐츠 이미지를 업데이트하지 않고, 그래프가 아닌 출력값만을 이용한다. 스타일 이미지 또는 콘텐츠 이미지를 업데이트 하지 않는다. 

출력과 그래프를 분리한 후 모든 추출 정보를 하나의 리스트 (targets)로 만든다. 

스타일 오차와 콘텐츠 오차를 계산할 때 content_layers 와 style_layers 2개의 배열을 전달했다. 다른 레이어를 선택하면 생성된 이미지의 품질에 영향을 미친다. 

옵티마이저는 단일 스칼라 값을 최소화한다. 하나의 스칼라 값을 얻기 위해 여러 레이어의 모든 오차를 합한다. 이 때 사용하는 일반적인 방법은 오차에 가중치를 적용한 합을 계산하는 것이다.

In [ ]:
style_weights = [1e3/n**2 for n in [64, 128, 256, 512, 512]]
content_weights = [1e0]
weights = style_weights + content_weights

In [ ]:
print(vgg)

### 각 레이어의 오차함수 만들기


In [ ]:
loss_fns = [StyleLoss()] * len(style_layers) + [nn.MSELoss()] * len(content_layers)

los_fns는 일련의 스타일 오차 객체와 콘텐츠 오차 객체를 포함하는 리스트이다. 스타일 오차 객체와 콘텐츠 오차 객체의 크기는 style_layer와 content_layer 배열 크기를 기준으로 만들어진다.

### 옵티마이저 만들기
일반적으로 신경망에서 학습시키는 대상은 네트워크 파라미터이지만 스타일 트랜스퍼에서는 VGG모델을 피처 추출기로 사용할 뿐이기 때문에 VGG 파라미터를 옵티마이저에 전달하지 않는다.

In [ ]:
from torch import optim

optimizer = optim.LBFGS([opt_img])

### 학습


In [ ]:
max_iter = 500
show_iter = 50
n_iter = [0]

while n_iter[0] <= max_iter:
    def closure():
        optimizer.zero_grad()
        out = extract_layers(loss_layers, opt_img, model=vgg)
        layer_losses = [weights[a] * loss_fns[a](A, targets[a]) for a,A in enumerate(out)]
        loss = sum(layer_losses)
        loss.backward()
        n_iter[0] += 1
        # 오차 출력
        if n_iter[0]%show_iter == (show_iter-1):
            print('Iteration:%d, loss:%f'%(n_iter[0]+1, loss.data[0]))
        
        return loss
        
    optimizer.step(closure)

extract_layers 함수를 호출해 vgg 모델로부터 여러 레이어 출력을 추출한다. 여기서 변경되는 유일한 것은 opt_img이다. opt_img에는 스타일 이미지를 포함한다. 출력이 추출되면 추출된 레이어를 순화하며 오차를 계산하고, 이 값을 해당 목표값과 함께 오차 함수에 전달한다. 이 오차를 모두 합산한 후에 backward 함수를 호출한다. closure 함수는 마지막에 이 오차를 반환한다. closure 함수는 max_iter만큼 반복하면서 optimizer.step 메서드와 함께 호출된다. 

## 생산적 적대 신경망 (GAN)

GAN은 위조 네트워크와 전문가 네트워크의 조합이다. 서로를 이기도록 훈련되는것. 생성기 네트워크는 입력으로 랜덤 벡터를 사용해 위조 이미지를 만든다. 판별 네트워크는 이미지를 입력하고, 그 이미지가 실제인지 가짜인지를 예측한다. 판별기 네트워크에는 실제 이미지가 입력될 수도 있고, 가짜 이미지가 입력될 수도 있다. 

생성기 네트워크는 이미지를 생성하고 판별기 네트워크를 속여 현실임을 믿도록 훈련된다. 판별기 네트워크는 속지 않으려 개선하며 생성기 네트워크에 피드백을 전달한다. 

## 심층 컨볼루션 GAN

DCGAN 논문을 기초로 GAN 아키텍처를 훈련시키는 여러 부분을 구현해본다.

DCGAN 학습에 필요한 요소는 다음과 같다.

> 생성기 네트워크 : 고정도니 차원의 잠재 벡터를 어떤 형상의 이미지에 대응시킨다. 예제에서는 (3,64,64)이다.

> 판별기 네트워크 : 생성기 네트워크가 만든 이미지 또는 실제 데이터 셋의 이미지를 입력받는다. 그리고 이미지가 실제인지 가자인지 평가 점수를 출력한다.

> 생성기와 판별기를 위한 오차 함수 정의

> 옵티마이저 정의

> GAN 학습


### 생성기 네트워크 정의

생성기 네트워크는 고정된 차원의 랜덤 벡터를 입력으로 사용한다. 랜덤 벡터에 전치 컨볼루션 배치 정규화 그리고 ReLU 활성화를 적용하고 필요한 크기의 이미지를 생성한다. 생성기 구현을 다루기 전에 전치 컨보루션 및 배치 정규화가 무엇인지 살펴보자


**전치 컨볼루션**

스트라이드 컨볼루션이라고도 불린다. 일반적인 컨볼루션이 동작하는 방식과 반대 방식으로 동작한다.

입력 벡터를 더 높은 차원에 매핑하는 방법을 계산하려 한다.

**배치 정규화**

머신 러닝 또는 딥 러닝 알고리즘에 전달되는 모든 피처는 정규화된다. 즉 피처 값에서 데이터의 평균을 빼고, 데이터의 표준 편차로 나눔으로써 데이터의 단위 표준 편차를 부여한다. 정규화 된 데이터는 0을 중심으로 집중하게 된다. 일반적으로 파이토치의 torchvision.Normalize 메서드를 사용해서 이 작업을 수행한다. 

    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

배치 정규화가 학습과정에서는 배치 데이터 단위로 평균과 분산이 계산된다.

검증, 테스트 단계에서는 전체 데이터의 평균과 분산값을 사용한다. 

매치 정규화를 사용하기 위해 이해해야할 것은 중간 데이터를 표준화하는것이다.

> 네트워크의 기울기 흐름이 개선되고, 더 깊은 네트워크를 구축할 수 있음.

> 더 큰 학습률을 사용할 수 있음.

> 초기화의 강한 의존성을 줄일 수 있음

> 정규화의 형태로 동작하고 드롭아웃의 의존성을 줄일 수 있음


배치 정규화 레이어(BN)은 컨볼루션 레이어 또는 선형, 전연결 레이어 다음에 위치한다.

**생성기 네트워크 구현 코드**

    from torch import nn

Generator 모델은 크기가 nz인 텐서를 입력으로 가져온 후 이를 전치 컨볼루션에 전달해 입력을 생성해야하는 이미지 크기에 대응시킨다.

forward 함수는 입력을 순차 모듈로 전달하고 출력을 반환한다.
    class Generator(nn.Module):
        def __init__(self):
            super(Generator, self).__init__()

            self.main = nn.Sequential(
                # 입력 : Z, 컨볼루션에 전달
                nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias = False),
                nn.BatchNorm2d(ngf * 8),
                nn.ReLU(Ture),
                # 상태 크기: (ngf*8) * 4 * 4
                nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf*4),
                nn.ReLU(True),
                # 상태 크기 : (ngf*4) * 8 * 8
                nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf*2),
                nn.ReLU(True),
                # 상태 크기 : (ngf*2) * 16 * 16
                nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf),
                nn.ReLU(True),
                # 상태 크기 : (ngf) * 32 * 32
                nn.ConvTranspose2d(ngf, nc,4,2,1, bias=False),
                nn.Tanh() 
Tanh 레이어는 Generator 네트워크가 생성하는 값의 범위를 한정
                # 상태 크기 : (nc) * 64 * 64

            )
        
        def forward(self, input):
            output = self.main(input)
            return output

    def weights_init(m):
        classname = m.__clas__.___name__
        if classname.find('Conv') != -1:
            m.weight.data.normal_(0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            m.weight.data.normal_(1.0, 0.02)
            m.bias.data.fill_(0)

    netG = Generator()
    netG.apply(weights_init)
    print(netG)
생성기 객체 Generator인 netGdml apply메서드에 weights_init 함수를 매개변수로 입력하면, 레이어 수 만큼 weights_init 함수가 호출된다.

weights_init 함수가 호출될 때, netG의 각 레이어가 함수의 전달 인자로 제공된다.

이때 입력된 렝이ㅓ가 컨볼루션레이어일 경우 평균이 0, 표준편차가 0.02가 되도록 초기화하고,

배치 정규화 레이어일 경우 편균이 1, 표준편차가 0.02가 되도록 초기화한다.
    


### 판별기 네트워크 정의

    class Discriminator(nn.Module):
        def __init__(self):
            super(_netD, self).__init__()
            self.main = nn.Sequential(
                # 입력 : (nc) * 64 * 64
                nn.Conv2d(nc, ndf, 4,2,1,bias=False),
                nn.LeakyReLU(0.2, inplace =True),
                # 상태 크기 : ndf * 32 * 32
                nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ndf * 2),
                nn.LeakyReLU(0.2, inplace=True),
                # 상태 크기 : (ndf*2) * 16 * 16
                nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ndf * 4),
                nn.LeakyReLU(0.2, inplace=True),
                # 상태 크기 : (ndf *4) * 8 * 8
                nn.Conv2d(ndf * 4, ndf*8, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ndf*8),
                nn.LeakyReLU(0.2, inplace=True),
                # 상태 크기 : (ndf*8) * 4 * 4
                nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
                nn.Sigmoid()

            )
        def forward(self, input):
            output = self.main(input)
            return output.view(-1, 1).squeeze(1)


    netD = Discriminator()
    netD.apply(weights_init)
    print(netD)

위 판별기는 네트워크에서 활성화 함수로 LeakyReLU를 사용하고, 마지막 활성화 함수로는 시그모이드를 사용한다.

**LeakyReLU**는 죽어가는 ReLU문제를 해결하기 위한 시도이다. LeakyReLU는 입력이 음수일 때 0을 반환하는 함수 대신 0.0001과 같은 매우 작은 숫자를 출력한다. GAN 논문에서 LeakyReLU를 사용하면서 판별기 네트워크의 성능이 향상되었다.

마지막 전연결 레이어를 사용하지 않았다. 
마지막으로 전연결 레이어가 전역 평균 풀링으로 대체되는 것이 일반적이지만 전역 평균 풀링을 사용하면 수렴 속도(정확한 분류자를 만드는데 필요한 반복 횟수)가 줄어든다. 마지마 컨볼루션 레이어는 평평해지고 시그모이드로 전달된다.


### 오차와 옵티마이저

    # 이진 교차 엔트로피 오차
    criterion = nn.BCELoss()

    # 옵티마이저 설정
    optimizerD = optim.Adam(netD.parameters(), lr, betas=(beta1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr, beats=(beta1, 0.999)) 

### 판별기 네트워크 학습

**실제 이미지로 판별기 학습시키기**

    output = netD(inputv)
    errD_real = criterion(output, labelv)
    errD_real.backward()

input 및 labelv는 CIFAR10 데이터셋 및 레이블의 입력 이미지의 실제 이미지이다.

**가짜 이미지로 판별기 학습시키기**

    fake = netG(noisev)
    output = netD(fake.detach())
    errD_fake = criterion(output, labelv)
    errD_fake.backward()
    optimizerD.step()

크기가 100인 벡터를 전달하고 생성기 네트워크(netG)는 이미지를 생성한다. 이미지가 실제인지 가짜인지 식별하기 위해 판별기 네트워크(netD)에게 전달한다. 판별기 네트워크가 훈련을 받ㅇ르 때 생성기 네트워크가 훈련 받기를 원하지 ㅇ낳기 때문에 가짜 이미지의 detach 메서드를 호출해서 그래프에서 가짜 이미지를 제거한다. backward 메서드를 호출해 도믄 기울기가 계산되면, 옵티마이저를 호출해 판별기 네트워크를 학습한다.

### 생성기 네트워크 학습

    netG.zero_grad()
    #가짜 레이블은 생성기 비용을 위해 실제 레이블로 만듦
    labelv = Variable(label.fill_(real_label))
    output = netD(fake)
    errG = criterion(output, labelv)
    errG.backward()
    optimizerG.step()

생성기 네트워크로 만든 가짜 이미지를 입력한다. 그러나 이번엔 생성기가 만든 그래프를 분리시키지 않는다. 왜냐하면 생성기가 훈련되길 원하기 때문이다. errG를 계산하고 errG 의 backward 메서드를 호출해 기울기를 계산한다. 생성기 옵티마이저를 호출해 생성기 네트워크를 훈련시킨다. 생성기가 이미지를 이미지를 생성하기 위해서는 이 전체 프로세스를 여러번 반복해야한다.

### 전체 네트워크 학습시키기
GAN 네트워크를 학습시키는 전체 코드를 살펴본다.

> 실제 이미지로 판별기 네트워크를 학습

> 가짜 이미지로 판별기 네트워크를 학습

> 판별기 네트워크 최적화

> 판별기 네트워크의 피드백을 기반으로 생성기 네트워크 학습

> 생성기 네트워크 최적화


In [ ]:
for epoch in range(niter):
    for i , data in enumerate(dataloader, 0):
        ################################################
        # (1) D 네트워크 업데이트 : maximize log(D(x)) + log(1-D(G(z)))
        ################################################
        # 실제 데이터로 훈련
        netD.zero_grad()
        real, _ = data
        batch_size = real.size(0)
        if torch.cuda.is_available():
            real = real.cuda()
        input.resize_as_(real).copy_(real)
        label.resize_(batch_size).fill_(real_label)
        inputv = Variable(input)
        labelv = Variable(label)

        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        # 가짜 데이터로 훈련
        noise.resize_(batch_size, nz, 1, 1).normal_(0,1)
        noisev = Variable(noise)
        fake = netG(nosiev)
        labelv = Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()

        ################################################
        # (2) G 네트워크 학습 : maximize log(D(G(z)))
        ################################################

        netG.zero_grad()
        # 가짜 레이블은 생성기 비용을 위해 실제 레이블로 만듦
        labelv = Variable(label.fill_(real_label))
        output = netD(fake)
        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()

        print('[%d/%d] [%d/%d] Loss_D:%.4f Loss_G:%.4f D(x):%.4f D(G(z)):%.4f /%.4f' % (epoch, niter, i, len(dataloader), errD.data[0],
                                                                                        errG.data[0], D_x, D_G_z1, D_G_z2))
        if i% 100 == 0:
            vutils.save_image(real_cpu, '%s/real_samples.png'%outf, normalize=True) 
            fake = netG(fixed_noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png'%(outf,epoch), normalize=True)

## 언어 모델

텍스트를 생성하는 방법을 순환 신경망(RNN)에 학습시켜본다!

### 데이터 준비

WikiText2라는 데이터셋을 사용.

In [ ]:
from torchtext import data as d
from torchtext import datasets

TEXT = d.Field(lower=True, batch_first=True)
train, valid, test = datasets.WikiText2.splits(TEXT, root='data')

WikiText2를 다운로드하고 train, valid, test로 데이터셋을 나누다.

WikiText2에 모든 텍스트 데이터는 하나의 긴텐서에 저장된다. 데이터가 더 잘 처리되는 방법을 이해하기 위해 다음 코드와 결과를 살펴보자.

In [ ]:
print(len(train[0].text))

In [ ]:
print(train[0].text[:100])

### 배치 처리기 생성

In [ ]:
train_iter, valid_iter, test_iter = d.BPTTIterator.splits(
    (train, valid, test), batch_size=20, bptt_len=35
)

위에서 가장 중요한 사항은 **batch_size**와 **bpt_len**이다. 

bptt는 "backpropagation through time"의 줄임말로 각 단계를 통해 데이터가 어떻게 변환되는지에 데한 간단한 아이디어를 제공한다.

**배치**
시퀀스를 한번에 처리하는 것은 비효율적이기 때문에 시퀀스 데이터를 배치로 나누고 각 시퀀스를 별도로 처리한다. 

**Backpropagation through time**
BPTT는 모델이 기억해야 하는 시퀀스의 길이를 의미한다. 숫자가 높을 수록 모델의 복잡성과 모델에 필요한 GPU 메모리가 증가한다.

### LSTM에 기반을 둔 모델 정의

In [ ]:
from torch import nn
from torch.autograd import Variable

class RNNModel(nn.Module):
    def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        #ntoken = 어휘의 단어의 수
        #ninp = LSTM에 입력되는 단어의 임베딩 차원
        #nlayer = LSTM에 사용될 레이어 수
        #Dropout = 과대적합 방지
        #tie_weights  = encoder와 decoder에 같은 가중치 사용 여부

        super().__init__()
        self.drop = nn.Dropout()
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(niinp, nhid, nlayers, dropout = dropout)
        self.decoder = nn.Linear(nhid, ntoken)
        if tie_weights:
            self.decoder.weight = self.encoder.weight
        self.init_weights()
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)
    
    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        s = output.size()
        decoded = self.decoder(output.view(s[0]*s[1], s[2]))
        return decoded.view(s[0], s[1], decoded.size(1)), hidden
    
    def init_hidden(self, bsz):
        weight = next(self.parameters()).data

        return (Variable(weight.new(self.nlayers, bsz, self.nhid).zero_())), Variable(weight.new(self.nlayers, bsz, self.nhid).zero_())

__init__ 메서드에서 embedding, dropout, RNN, decoder와 같은 모든 레이어를 생성한다. 초기 언어 모델에서는 임베딩 레이어가 일반적으로 마지막 레이어 에서 사용되지 않았다. 임베딩을 사용하고 최종 임베디드를 최종 출력 레이어의 임베딩과 연결하면 언어 모델의 정확성이 향상된다. 인코더와 디코더의 가중치를 묶었으면, init_weights메서드를 호출해 레이어의 가중치를 초기화한다.

forward 함수는 모든 레이어의 흐름을 정의한다. 마지막 선형 레이어는 LSTM 레이어의 출력을 어휘 크기의 임베딩에 대응시킨다. forward 함수에 입력 데이터의 흐름은 임베딩 레이어를 통과한 후 RNN으로 전달되고, 마지막으로 선형 레이어인 디코더에 전달된다.


### 학습과 평가 함수 정의

In [ ]:
criterion = nn.CrossEntropyLoss()

def trainf():
    # 드롭아웃을 가능하게 하는 트레이닝 모드를 켠다.
    lstm.train()
    total_loss = 0
    start_time = time.time()
    hidden = lstm.init_hidden(batch_size)
    for i, batch in enumerate(train_iter):
        data, targets = batch.text, batch.target.view(-1)
        # 각 배치를 시작하고, 숨겨진 상태를 이전에 생성된 상태에서 분리
        # 그렇지 않은 경우 모델은 데이터셋을 시작하기 위해 역전파를 시도
        hidden = repackage_hidden(hidden)
        lstm.zero_grad()
        output, hidden = lstm(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # 'clip_grad_norm'은 RNNs / LSTM 의 기울기 폭발을 방지
        torch.nn.utils.clip_grad_norm(lstm.parameters(), clip)
        for p in lstm.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.data

        if i% log_interval == 0 and i>0:
            cur_loss = total_loss[0] / log_interval
            elapsed = time.time() - start_time
            (print('| epoch {:3d} | {:5d} / {:5d} batches | lr {:02.2f} | ms/batch {5.2f} | loss {:5.2f} | ppl {8.2f}'.format(epoch, i, len(train_iter), lr, elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss))))
            total_loss = 0
            start_time = time.time()

모델에서 드롭아웃을 사용하기 때문에 훈련 및 검증 데이터셋에서 다르게 사용해야 한다. 모델에서 train()을 호출하면 학습 중에 드롭아웃이 활성화된다. 모델의 eval()을 호출하면 드롭아웃이 비활성화된다.

    lstm.train()

LSTM 모델의 경우 입력과 함게 hidden 변수도 전달해야 한다. init_hidden 함수는 배치 크기를 입력으로 가져온 후, 입력과 함께 사용할 수 있는 hidden 변수를 반환한다. 학습 데이터를 반복하고 입력 데이터를 모델에 전달할 수 있다. 시퀀스 데이터를 처리하기 때문에 모든 반복에 대해 새로운 히든 상태(랜덤으로 초기화된)로 시작하는 것이 의미가 없다. 그래서 detach 메서드를 호출해 그래프에서 제거한 후 이전 반복에서 숨겨진 상태를 사용한다. detach 메서드를 호출하지 않으면 GPU 메모리가 부족할 때까지 매우 긴 시퀀스의 기울기를 계산하게 된다.

LSTM 모델의 입력을 전달하고 Cross EntropyLoss를 사용해 오차를 계산한다. 숨겨진 상태의 이전 값을 사용하는 방법은 repackage_hidden 함수에서 구현된다.



In [ ]:
def repackage_hidden(h):
    """새로운 변수에 hidden 상태를 래핑한다."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

RNN과 LSTM 및 Gated Recurrent Unit 과 같은 변형은 기울기 폭발 문제로 어려움을 겪는다. 문제를 피하는 간단한 방법은 기울기를 자르는 것이다. 

    torch.nn.utils.clip_grad_norm(lstm.parameters(), clip)

매개변수의 값을 수동으로 조정한다. 옵티마이저를 수동으로 구현하면 미리 작성된 옵티마이저를 사용하는 것보다 더 많은 유연성을 얻을 수 있다.

    for p in lstm.parameters():
    p.data.add(-lr, p.grad.data)

우리는 모든 매개변수를 반보갛고 기울기 값에 학습율을 곱한 값을 더한다. 모든 매개변수를 업데이트 하면 시간, 오차, 및 복잡성 같은 모든 통계가 기록된다.

검증을 위해 비슷한 함수를 하나 더 작성한다. 여기서는 모델이서 eval 메서드를 호출한다. evalutate 함수는 다음 코드를 사용해 정의한다.

In [ ]:
def evaluate(data_source):
    # 드롭아웃을 비활성화하는 평가 모드를 켠다.